In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jan 14 10:42:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   39C    P0    52W / 300W |  10969MiB / 16384MiB |     67%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import transformers
print(transformers.__version__)

4.11.3


In [3]:
import os

In [4]:
import sys

In [5]:
#!{sys.executable} -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio===0.12.1+cu113 -f https://download.pytorch.org/whl/torch_stable.html

# this code worked with the following torch setup
#!{sys.executable} -m pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
import torch
import torchvision
torch.cuda.get_device_name()

'Tesla V100-SXM2-16GB'

In [ ]:
!{sys.executable} -m pip install numpy==1.20.3

In [ ]:
!{sys.executable} -m pip install datasets==2.0.0
!{sys.executable} -m pip install transformers==4.11.3
!{sys.executable} -m pip install torchaudio
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install jiwer


In [ ]:
#!{sys.executable} -m pip install ipywidgets

In [ ]:
#!{sys.executable} -m jupyter nbextension enable --py widgetsnbextension

In [8]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/ubuntu/.huggingface/token
Login successful


In [9]:
from datasets import load_dataset

fleurs_asr = load_dataset("google/fleurs", "ln_cd")  # for Lingala
# to download all data for multi-lingual fine-tuning uncomment following line
# fleurs_asr = load_dataset("google/fleurs", "all")

# see structure
print(fleurs_asr)

# load audio sample on the fly
audio_input = fleurs_asr["train"][0]["audio"]  # first decoded audio sample
transcription = fleurs_asr["train"][0]["transcription"]  # first transcription
# use `audio_input` and `transcription` to fine-tune your model for ASR

# for analyses see language groups
all_language_groups = fleurs_asr["train"].features["lang_group_id"].names
lang_group_id = fleurs_asr["train"][0]["lang_group_id"]

all_language_groups[lang_group_id]

Reusing dataset fleurs (/home/ubuntu/.cache/huggingface/datasets/google___fleurs/ln_cd/2.0.0/aabb39fb29739c495517ac904e2886819b6e344702f0a5b5283cb178b087c94a)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 3350
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 209
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 478
    })
})


'sub_saharan_african_ssa'

In [10]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [11]:
fleurs_asr_train = fleurs_asr['train'].remove_columns(["id", "num_samples", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])
fleurs_asr_test = fleurs_asr['validation'].remove_columns(["id", "num_samples", "raw_transcription", "gender", "lang_id", "language", "lang_group_id"])

In [12]:
show_random_elements(fleurs_asr_train.remove_columns(["path", "audio"]), num_examples=10)

,transcription
0,"vitese ya 802,11n mbala mingi eza makasi koleka ya oyo ya kala po ezalaki na vitese ya ebele koleka ya 600mbit/s"
1,elongo na bakonzi wana andimisaki bato ya texas ete bazozwa bibongiseli mpo bato nyonso bazala na libateli
2,bandimaka livres sterling na bisika nyonso na bisanga wana na crédit stanley mpe mbala mingi bandimaka badolare ya etats-unis
3,bisanga cook eza te na engumba ata moko kasi eza na bisanga 15 ekeseni. bisanga ya minene eza rarotonga na aitutaki
4,mobeko wana ezosenga ete masano ya video ya mbulu oyo bazoteka na etuka ya californie bakoma likolo ya ngo ete 18 pe emonisa polele ete soki batekeli yango bana-mike bakofunda amande ya 1000$
5,soki okei kotala groenland na bileko ya malili yeba ete soki ozokoba kokende na nord malili ekozala makasi koleka eza ntina malamu komema biloko ya molunge
6,tornade eza molongo ya mopepe oyo ezobaluka na pression ya nse ebendaka mopepe ya zingazinga na kati mpe na libanda
7,modele esengeli kozala mozindo 5 mm 1/5 inch to mingi mpe biloko esengeli kozala mwa pete mpona bisika ya malili makasi
8,na bamboka mosusu na ndakisa na etats-unis mpe na canada balongolaka mpako na salere na leta pe na mboka yango wana motango yango ekoki kokesena na etuka mokomoko
9,eloko ya up bow emonanaka neti v pe oyo ya down bow mark neti agrafe to carré ezangi nzela ya se


In [13]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\[\]\¥\+\ł\£\—\½\~\$\&\¾\²\°\/\–]'

def remove_special_characters(batch):
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [14]:
fleurs_asr_train = fleurs_asr_train.map(remove_special_characters)
fleurs_asr_test = fleurs_asr_test.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f42e58b0af0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/3350 [00:00<?, ?ex/s]

  0%|          | 0/209 [00:00<?, ?ex/s]

In [15]:
show_random_elements(fleurs_asr_test.remove_columns(["path","audio"]))

,transcription
0,banzela ya mai ekoki kozala likanisi malamu mpona kobongisa conge
1,robin uthappa azwaki ba point eleki mingi na toure ya liboso ba point 70 na ba ballon 41 pamba ebetaki 11 quatre na 2 six
2,bangando ya mai mungwa bafandaka mpenza te na mbu kasi esika na bango ya libosoliboso ya kofanda ezala na ebale na nordi ya rockhampton
3,bato ya portugal babomaki yango mpe batongaki yango na nkombo ya casa branca na nsima babwakisaki yango nsima ya koningana ya mabele na 1755
4,badepanse ya bitumba ya baniokoli ya etatsunis mpona kobotola philippines efutamaki na bato ya philippines bango moko
5,bizaleli oyo ekesenisaka bokoko ya se ekoki kozala makambo ya lokota esthétiques losambo politiki kosangisa nzoto géographiques to bosangani ya makambo mingi
6,alobaki ete asalaki eloko esalaka makelele ya wifi
7,esengaka katikati ya ngonga moko mpona kotambola na mboka kitoko
8,eza na bopusi mingi ya social pe ya politiki na ndakisa kosalela systeme ya metele kolongola absolutisme pe kosalela républicanisme kolongola nationalisme mpe kondima ete mboka ezali ya bato nyonso kasi ya mokonzi moko te
9,siansi emonisi sikoyo ete mombongo wana monene ya carbone elongoli biosphère na moko ya bituka ya malamu oyo esungaki bokoli ya bato na eleko ya mibu bamilio mibale oyo eleki


In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [17]:
vocab_train = fleurs_asr_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fleurs_asr_train.column_names)
vocab_test = fleurs_asr_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=fleurs_asr_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [19]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'s': 0,
 'n': 1,
 '3': 2,
 'f': 3,
 'z': 4,
 'p': 5,
 'u': 6,
 'd': 7,
 "'": 8,
 '5': 9,
 'm': 10,
 'j': 11,
 'r': 12,
 'w': 13,
 '1': 14,
 '0': 15,
 'c': 16,
 'í': 17,
 'q': 18,
 'a': 19,
 'l': 20,
 'ó': 21,
 ' ': 22,
 'b': 23,
 'e': 24,
 'ü': 25,
 '8': 26,
 'ç': 27,
 'i': 28,
 '4': 29,
 'è': 30,
 'h': 31,
 'ñ': 32,
 '7': 33,
 'â': 34,
 'ï': 35,
 'á': 36,
 '2': 37,
 'k': 38,
 '9': 39,
 '6': 40,
 'œ': 41,
 'é': 42,
 'ã': 43,
 'o': 44,
 'x': 45,
 'y': 46,
 'à': 47,
 'ö': 48,
 'ú': 49,
 'v': 50,
 'î': 51,
 'g': 52,
 't': 53,
 'õ': 54}

In [20]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [21]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

57

In [22]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [24]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [25]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [26]:
fleurs_asr_train[0]["path"]

'/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/21fd69d4c7cc2f3f10045659e795a2252a81b7a8dc04b6d59e3bba5e20de9ffc/ln_cd/audio/train/9356927913403290682.wav'

In [27]:
fleurs_asr_train[0]["audio"]

{'path': '9356927913403290682.wav',
 'array': array([0.        , 0.        , 0.        , ..., 0.0061636 , 0.00686049,
        0.00756782]),
 'sampling_rate': 16000}

In [28]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(fleurs_asr_train)-1)

ipd.Audio(data=fleurs_asr_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

In [29]:
#rand_int = random.randint(0, len(fleurs_asr_train)-1)

print("Target text:", fleurs_asr_train[rand_int]["transcription"])
print("Input array shape:", fleurs_asr_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", fleurs_asr_train[rand_int]["audio"]["sampling_rate"])

Target text: bato ebele balotaka mokolo oyo bato bakoki kosala mobembo na monzoto mosusu pe komona mikili mosusu bato mosusu bamitunaka nini eza kuna basusu bakanisaka bapaya to bikelamu mosusu efandaka na planete mosusu 
Input array shape: (534720,)
Sampling rate: 16000


In [30]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["transcription"]).input_ids
    return batch

In [31]:
fleurs_asr_train = fleurs_asr_train.map(prepare_dataset, remove_columns=fleurs_asr_train.column_names, num_proc=4)
fleurs_asr_test = fleurs_asr_test.map(prepare_dataset, remove_columns=fleurs_asr_test.column_names, num_proc=4)

#0:   0%|          | 0/838 [00:00<?, ?ex/s]

#1:   0%|          | 0/838 [00:00<?, ?ex/s]

#2:   0%|          | 0/837 [00:00<?, ?ex/s]

#3:   0%|          | 0/837 [00:00<?, ?ex/s]

#0:   0%|          | 0/53 [00:00<?, ?ex/s]

#1:   0%|          | 0/52 [00:00<?, ?ex/s]

#2:   0%|          | 0/52 [00:00<?, ?ex/s]

#3:   0%|          | 0/52 [00:00<?, ?ex/s]

In [32]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [33]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [34]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [35]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [36]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "Ussen/congowav2vec-multilingual_pretrained", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at Ussen/congowav2vec-multilingual_pretrained were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.bias', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Ussen/congowav2vec-multilingual_pretrained and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-strea

In [37]:
#from transformers import Wav2Vec2ForCTC

#model = Wav2Vec2ForCTC.from_pretrained(
#    "facebook/wav2vec2-large-xlsr-53", 
#    attention_dropout=0.1,
 #   hidden_dropout=0.1,
 #   feat_proj_dropout=0.0,
 #   mask_time_prob=0.05,
 #   layerdrop=0.1,
 #   ctc_loss_reduction="mean", 
 #   pad_token_id=processor.tokenizer.pad_token_id,
 #   vocab_size=len(processor.tokenizer)
#)

In [38]:
model.freeze_feature_extractor()

In [39]:
model.gradient_checkpointing_enable()

In [40]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2",
  group_by_length=False,
  per_device_train_batch_size=1,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  per_device_eval_batch_size=4,
  num_train_epochs=30,
  fp16=True,
  save_steps=300,
  eval_steps=300,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [41]:
from huggingface_hub import create_repo, Repository

import logging
import sys
repo_url = create_repo("cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2")

In [42]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=fleurs_asr_train,
    eval_dataset=fleurs_asr_test,
    tokenizer=processor.feature_extractor,
)

Cloning https://huggingface.co/Ussen/cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2 into local empty directory.
Using amp fp16 backend


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3350
  Num Epochs = 30
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 25110


Step,Training Loss,Validation Loss,Wer
300,2.986100,2.770642,1.000000
600,2.809600,2.813446,1.000000
900,2.817200,2.763016,1.000000
1200,2.795800,2.767812,1.000000
1500,2.803100,2.757329,1.000000
1800,2.800600,2.771496,1.000000
2100,2.789600,2.752253,1.000000
2400,2.797200,2.779482,1.000000
2700,2.801300,2.763247,1.000000
3000,2.799000,2.755735,1.000000


***** Running Evaluation *****
  Num examples = 209
  Batch size = 4
Saving model checkpoint to ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-300
Configuration saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-300/config.json
Model weights saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-300/pytorch_model.bin
Configuration saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-300/preprocessor_config.json
Configuration saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 209
  Batch size = 4
Saving model checkpoint to ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-600
Configuration saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoint-600/config.json
Model weights saved in ./cdwav2vec2-multilingual-finetuned-on-google-fleurs-ln_cd2/checkpoi

In [ ]:
kwargs = {
    "dataset_tags": "Google__fleurs(ln_cd subset)",
    "dataset": "Google__fleurs(ln_cd subset)",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ln, split: validation",
    "language": "ln",
    "model_name": "Ussen",  # a 'pretty' name for your model
    "finetuned_from": "Ussen/congowav2vec-multilingual_pretrained",
    "tasks": "automatic-speech-recognition",
    "tags": "ln-asr-leaderboard",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [9]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("/home/ubuntu/wav2vec2-large-xlsr-lingala-fleurs2/checkpoint-24800/")

In [12]:
from huggingface_hub import create_repo, Repository

import logging
import sys
repo_url = create_repo("wav2vec2-xlsr-finetuned-on-google-fleurs-ln_cdfff")

In [10]:
model.push_to_hub()

ValueError: You need to specify a `repo_path_or_name` or a `repo_url`.